In [2]:
import numpy as np
from numpy import linalg as LA
import sympy as sp
import math
import copy

In [3]:
#This fucntion adds up the weights which is to be used as an inital guess for the required eigenvalye.
def sumweights(w,k):
    temp = 0.0
    for i in range(k):
        temp = temp + w[i]
    return (temp)

In [4]:
#The next five functions calculate components required to build a matrix 'K', crucial for further algorithm.

#This function calculates a matrix 'B', a summation of outerproducts of vectors in both frames scaled to their weights.
def B(r,b,w,k):
    Bmatrix = np.array([[0,0,0],[0,0,0],[0,0,0]])
    for i in range(k):
        Bmatrix = (Bmatrix+w[i]*(np.array(b[i]).T)@np.array(r[i]))
    return (Bmatrix)

#This function calculates the trace of the above obtained matrix
def sigma(r,b,w,k):
    return (np.trace(B(r,b,w,k)))

#This function calculates a matrix 'S' which is nothing but sum of 'B' and its transpose.
def S(r,b,w,k):
    return (B(r,b,w,k)+B(r,b,w,k).T)

#Defines a matrix which is nothing but difference of 'S' matrix and a scalar matrix with diagonal elements as the trace of 'B'.
def S2(r,b,w,k):
    return (S(r,b,w,k)-sigma(r,b,w,k)*np.array([[1,0,0],[0,1,0],[0,0,1]]))

#Defines a column matrix 'Z' from the elements of matrix 'B'
def Z(r,b,w,k):
    A = B(r,b,w,k)
    return ((np.array([[A[1,2]-A[2,1],A[2,0]-A[0,2],A[0,1]-A[1,0]]]).T))

In [5]:
#Defines a matrix 'K' as mentioned earlier using the returned quantities from the above function.
def K(r,b,w,k):
    sigmatemp = sigma(r,b,w,k)
    Ztemp = Z(r,b,w,k)
    S2temp = S2(r,b,w,k)
    A = np.array([[sigmatemp,Ztemp[0,0],Ztemp[1,0],Ztemp[2,0]],[Ztemp[0,0],S2temp[0,0],S2temp[0,1],S2temp[0,2]],[Ztemp[1,0],S2temp[1,0],S2temp[1,1],S2temp[1,2]],[Ztemp[2,0],S2temp[2,0],S2temp[2,1],S2temp[2,2]]])
    return (A)

In [6]:
def Eigen(r,b,w,k):
    Atemp = sp.Matrix(K(r,b,w,k))
    return LA.eig(K)

In [7]:
#Give the value of the 'characterisitic polynomial' of matrix 'K' when a value of 's' is substituted.
def F(r,b,w,k,s):
    Ktemp = K(r,b,w,k)-np.array([[s,0,0,0],[0,s,0,0],[0,0,s,0],[0,0,0,s]])
    return (sp.det(sp.Matrix(Ktemp)))

In [8]:
#Gives the value of the derivative of the above polynomial when a value of 's' is substituted.
def Fprime(r,b,w,k,s):
    m = sp.Symbol('m')
    Ktemp = sp.Matrix(K(r,b,w,k))-sp.Matrix([[m,0,0,0],[0,m,0,0],[0,0,m,0],[0,0,0,m]])
    func = sp.det(Ktemp)
    deri = sp.diff(func,m)
    return (deri.subs({m:s}))

In [9]:
#Calculates a pretty close root to the 'characterisitic polynomial' of matrix 'K' using Newton Raphson's method.
#Roots to a characteristic polynomial of a matrix are its eigenvalues.
def root(r,b,w,k,s):
    a0 = s+0.0
    while F(r,b,w,k,a0)>0.01:
        a0 = a0 - (F(r,b,w,k,a0)/Fprime(r,b,w,k,a0))
    return (a0)

In [10]:
#Determines the 'classical rodrigues parameter'.
def crp(r,b,w,k):
    u = root(r,b,w,k,sumweights(w,k)) + sigma(r,b,w,k)
    Stemp = S(r,b,w,k)
    Ztemp = Z(r,b,w,k)
    p = np.array([[u,0,0],[0,u,0],[0,0,u]])-Stemp
    return ((sp.Matrix(p)**-1)@Ztemp)

In [11]:
#Calculates the 'quaternion' form of the attitude, (Convention used: First element is the scalar, and next three is the vector)
def quaternion(r,b,w,k):
    qtemp = crp(r,b,w,k)
    a = 1/(math.sqrt(1+sp.det((qtemp.T)*qtemp)))
    return (np.array(sp.Matrix([[a],[a*qtemp[0,0]],[a*qtemp[1,0]],[a*qtemp[2,0]]])))

In [12]:
#Takes the nummber of measurements
k = int(input("Please enter the number of measurements: "))

Please enter the number of measurements: 2


In [13]:
#Takes the vectors in reference frame
print ("Please enter the unit vectors for them in the 'Reference frame': ")
empty = [[]]
r = []
for i in range(k):
    r.append(copy.copy(empty))
for i in range(k):
    r[i][0] = list(map(float,(input(str(i+1)+": ").split())))

Please enter the unit vectors for them in the 'Reference frame': 
1: 1 0 0
2: 0 1 0


In [14]:
#Takes the vectors in body frame
print ("Please enter the unit vectors for them in the 'Body frame': ")
b = []
for i in range(k):
    b.append(copy.copy(empty))
for i in range(k):
    b[i][0] = list(map(float,(input(str(i+1)+": ").split())))

Please enter the unit vectors for them in the 'Body frame': 
1: 0 -1 0
2: 1 0 0


In [15]:
#Takes the weights of each vector measurement
w = []
for i in range(k):
    w.append(float(input(str(i+1)+": ")))

1: 1
2: 1


In [16]:
#Prints quaternion form of Attitude
print (quaternion(r,b,w,k))


[[0.707106781186547]
 [0]
 [0]
 [0.707106781186547]]
